# 기업 시장/재무 데이터 분석 

## 1) 기업 종목 코드 크롤링 

### 1-1) 한국거래소 데이터를 바로 크롤링

밑의 코드를 사용하면 현재 한국거래소에 상장되어있는 기업들의 기업명, 종목코드를 간편하게 수집 가능.  
하지만 이 방법으로는 업종코드를 따로 붙여야함.  

In [1]:
# 한국거래소에서 상장기업정보 받기 

import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명','종목코드']]
code_df = code_df.rename(columns={'회사명':'name','종목코드':'code'})

code_df.head()

,name,code
0,세이브존I&C,067830
1,동아쏘시오홀딩스,000640
2,현대건설기계,267270
3,웅진,016880
4,삼영화학공업,003720


### 1-2) 한국거래소에서 csv 파일 다운로드 

한국거래소에서 다운받은 상장기업정보 csv파일을 dataframe으로 읽어옴.  
업종코드가 붙어있어서 편하지만, 직접 csv파일을 다운받아야하는 번거로움.  

In [17]:
# 한국거래소에서 다운받은 csv파일 데이터프레임으로 저장하기 
#-*- coding: utf-8 -*-

corp_df = pd.read_csv('corp_code2.csv')
corp_df.head()

,code,name,cate_code,cate_name
0,60310,3S,32902,특수 목적용 기계 제조업
1,95570,AJ네트웍스,147603,산업용 기계 및 장비 임대업
2,68400,AJ렌터카,147601,운송장비 임대업
3,6840,AK홀딩스,116409,기타 금융업
4,54620,APS홀딩스,116409,기타 금융업


In [19]:
# 상장된 기업 2208개 확인 

len(corp_df)

2208

## 2) 기업 재무 데이터 크롤링 

네이버 금융 페이지 웹크롤링 


In [21]:
# 필요한 모듈 임포트 

import re
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
# 컬럼명 정리 함수 생성 

def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
    if r:
        date_str = r.group()
        date_str = date_str.replace('/', '-')
    return date_str

In [67]:
# 종목코드 인자로 넣으면 분기별 재무정보 반환 함수 
'''
* code: 종목코드
* fin_type = '0': 재무제표 종류 (0: 주재무제표, 1: GAAP개별, 2: GAAP연결, 3: IFRS별도, 4:IFRS연결)
* freq_type = 'Y': 기간 (Y:년, Q:분기)
'''

def get_finstate_naver(code, fin_type='0', freq_type='Q'):

    # 종목코드에 따른 url 생성 
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?' \
                   'cmp_cd=%s&fin_typ=%s&freq_typ=%s'
    url = url_tmpl % (code, fin_type, freq_type)

    # 재무정보 크롤링 
    dfs = pd.read_html(url, encoding="utf-8")
    df = dfs[0]
    if df.ix[0,0].find('해당 데이터가 존재하지 않습니다') >= 0:
        return None

    # 컬럼명 정리 
    df.rename(columns={'주요재무정보':'date'}, inplace=True)
    df.set_index('date', inplace=True)
    
    cols = list(df.columns)
    cols = [get_date_str(cols[i][1]) for i in range(len(cols))]
    df.columns = cols
    df = df.ix[:, :-1]

    # 로우명 정리 
    index = list(df.index)
    adj_index = [df.index[i][0] for i in range(len(index))]
    adj_index
    df.index = adj_index
    df.head()
    
    # 재무정보가 컬럼명으로 들어가 분석에 용이하게끔 원df의 전치행렬 생성 
    dft = df.T
    
    # date 데이터타입 변경 
    dft.index = pd.to_datetime(dft.index)

    # remove if index is NaT
    dft = dft[pd.notnull(dft.index)]
    
    # 코드명 컬럼 생성
    dft['code'] = code
    
    return dft

get_finstate_naver('035720')

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주),code
2017-06-01,4438.0,383.0,383.0,182.0,545.0,537.0,8.0,54946.0,17477.0,37470.0,...,10177.78,794.0,57.16,51263.0,1.62,0.0,0.0,0.00,67716172.0,035720
2017-09-01,4684.0,446.0,446.0,360.0,125.0,113.0,12.0,58101.0,18179.0,39922.0,...,10591.00,167.0,68.18,53379.0,1.90,0.0,0.0,0.00,67781135.0,035720
2017-12-01,5154.0,474.0,474.0,631.0,399.0,321.0,78.0,64941.0,20489.0,44452.0,...,11609.17,473.0,80.89,58640.0,2.46,0.0,NaN,0.00,67839099.0,035720
2018-03-01,5447.0,350.0,350.0,360.0,182.0,115.0,67.0,63494.0,18416.0,45078.0,...,11781.26,170.0,85.50,59336.0,2.31,148.0,NaN,87.21,67908527.0,035720
2018-06-01,5554.0,104.0,104.0,372.0,146.0,139.0,8.0,77249.0,19727.0,57522.0,...,13534.08,187.0,133.26,67954.0,1.94,0.0,NaN,0.00,76273522.0,035720
2018-09-01,5891.0,307.0,NaN,425.0,293.0,266.0,NaN,75829.0,18598.0,57231.0,...,NaN,348.0,NaN,67447.0,1.75,NaN,NaN,NaN,NaN,035720
2018-12-01,6119.0,394.0,NaN,515.0,360.0,367.0,NaN,82820.0,18451.0,64370.0,...,NaN,481.0,NaN,75707.0,1.56,NaN,NaN,NaN,NaN,035720


In [81]:
# 2208개의 상장기업 재무정보 데이터프레임 union join
test = corp_df.head(5)

final = pd.DataFrame()

for i in test.code :
    final = pd.concat([get_finstate_naver(i),final])
    

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [82]:
final

""


In [83]:
len(final)

0